In [1]:
import pandas as pd
import numpy as np
import datetime
import os

os.getcwd()

'/home/jian/Projects/Savers/Store_of_zips_already_selected_BR_20190531'

In [2]:
excel_file=pd.ExcelFile("/home/jian/Projects/Savers/Store_of_zips_already_selected_BR_20190531/Savers 2019 Q2 Q3 Audience Cluster Zip 20190530.xlsx")
excel_file.sheet_names

['Summary',
 'USA TA_zips',
 'CAN TA_zips De-Duped',
 'US_C1_NonExpose_CELLA',
 'US_C1_Expose_CELLB',
 'US_C2-4_NonExpose_CELLC',
 'US_C2-4_Expose_CELLD',
 'CAN_C2_NonExposeCELLE',
 'CAN_C2_Expose_CELLF',
 'CAN_C1.3.4_NonExpose_CELLG',
 'CAN_C1.3.4_Expose_CELLH']

In [4]:
US_zips_C=excel_file.parse("US_C2-4_NonExpose_CELLC",dtype=str)
US_zips_D=excel_file.parse("US_C2-4_Expose_CELLD",dtype=str)

CAN_zips_G=excel_file.parse("CAN_C1.3.4_NonExpose_CELLG",dtype=str)
CAN_zips_H=excel_file.parse("CAN_C1.3.4_Expose_CELLH",dtype=str)

if "Store" in US_zips_C.columns:
    del US_zips_C['Store']
    
if "Store" in US_zips_D.columns:
    del US_zips_D['Store']

if "Store" in CAN_zips_G.columns:
    del CAN_zips_G['Store']

if "Store" in CAN_zips_H.columns:
    del CAN_zips_H['Store']

In [5]:
USA_all_zips=excel_file.parse("USA TA_zips",dtype=str)
CAN_all_zips=excel_file.parse("CAN TA_zips De-Duped",dtype=str)

In [6]:
USA_all_zips=USA_all_zips.groupby("zip_cd")['location_id'].apply(set).to_frame().reset_index()
CAN_all_zips=CAN_all_zips.groupby("zip 3")['location_id'].apply(set).to_frame().reset_index()

USA_all_zips['count_of_associated_stores']=USA_all_zips['location_id'].apply(lambda x: len(x))
CAN_all_zips['count_of_associated_stores']=CAN_all_zips['location_id'].apply(lambda x: len(x))


In [7]:
us_max_store_count=max(USA_all_zips['location_id'].apply(lambda x: len(x)).unique())
ca_max_store_count=max(CAN_all_zips['location_id'].apply(lambda x: len(x)).unique())




In [8]:
for i in range(us_max_store_count):
    USA_all_zips['store_'+str(i+1)]=np.nan
for i in range(ca_max_store_count):
    CAN_all_zips['store_'+str(i+1)]=np.nan

In [9]:
for index,row in USA_all_zips.iterrows():
    store_list=sorted(list(row['location_id']))
    list_filled_na=store_list+[np.nan]*(us_max_store_count-len(store_list))
    for i in range(us_max_store_count):
        USA_all_zips.loc[index,"store_"+str(i+1)]=list_filled_na[i]
        
        
for index,row in CAN_all_zips.iterrows():
    store_list=sorted(list(row['location_id']))
    list_filled_na=store_list+[np.nan]*(ca_max_store_count-len(store_list))
    for i in range(ca_max_store_count):
        CAN_all_zips.loc[index,"store_"+str(i+1)]=list_filled_na[i]
CAN_all_zips=CAN_all_zips.rename(columns={"zip 3":"zip_cd"})    

In [10]:
US_zips_C=pd.merge(US_zips_C,USA_all_zips,on="zip_cd",how="left")
US_zips_D=pd.merge(US_zips_D,USA_all_zips,on="zip_cd",how="left")

CAN_zips_G=pd.merge(CAN_zips_G,CAN_all_zips,on="zip_cd",how="left")
CAN_zips_H=pd.merge(CAN_zips_H,CAN_all_zips,on="zip_cd",how="left")


In [13]:
writer=pd.ExcelWriter("./Savers_zips_with_respective_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

US_zips_C.to_excel(writer,"US_zips_C",index=False)
US_zips_D.to_excel(writer,"US_zips_D",index=False)
CAN_zips_G.to_excel(writer,"CAN_zips_G",index=False)
CAN_zips_H.to_excel(writer,"CAN_zips_H",index=False)

writer.save()